In [1]:
import qforte as qf
from qforte import *
import numpy as np

$$
\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
\newcommand{\cre}[1]{\hat{a}^{\dagger}_{#1}}
\newcommand{\ann}[1]{\hat{a}_{#1}}
$$

# Running jobs with QForte

In this tutorial we are interested in utilizing QForte as a tool to run black-box calculations. We will provide a theoretical overview and example code for how to run multiple quantum algorithms.

## Get molecular Hamiltonain

As we have done before in previous tutorials, we will need to first obtain the QForte `molecule` object for the system of interest. In this tutorial we will consider LiH.  

In [2]:
# Define the reference and geometry lists.
geom = [('Li', (0., 0., 0.0)), ('H', (0., 0., 1.50))]

# Get the molecule object that now contains both the fermionic and qubit Hamiltonians.
LiHmol = system_factory(build_type='psi4', mol_geometry=geom, basis='sto-3g', run_fci=True)

 ==> Psi4 geometry <==
-------------------------
0  1
Li  0.0  0.0  0.0
H  0.0  0.0  1.5
symmetry c1
units angstrom

  Memory set to   1.863 GiB by Python driver.


## QForte algorithms

QForte contains black-box implementations of the following algorithms:

- Disentangled (Trotterized) unitary coupled cluster variational quantum eigensolver (dUCCVQE)
  - QForte will treat up to hex-tuple particle-hole excitations (SDTQPH) or generalized singled and doubles (GSD).


- Adaptive derivative-assembled pseudo Trotterized VQE (ADAPT-VQE).
  
  
- Disentangled (Trotterized) unitary coupled cluster projective quantum eigensolver (dUCCPQE)
  - QForte will treat up to hex-tuple particle-hole excitations (SDTQPH).
  
  
- Selected projective quantum eigensolver (SPQE)


- Single reference Quantum Krylov (SRQK)


- Multireference selected quantum Krylov diagonalization (MRSQK)


- Quantum imaginary time evolution (QITE)


- Quantum Lanczos (QL)


- Quantum phase estimation (QPE)

### Example 1:  dUCC-VQE

Similarly to the dUCC-PQE example in tutorial 5, dUCC-VQE likewise considers a factorized (disentangled) form of the UCC ansatz
\begin{equation}
\hat{U}(\mathbf{t})=
 \prod_\mu e^{ t_\mu \hat{\kappa}_\mu}.
\end{equation}

The VQE energy expectation value is then given as
\begin{equation}
E_\text{VQE}(\mathbf{t}) = \bra{\Phi_0} \hat{U}^\dagger(\mathbf{t}) \hat{\mathcal{H}} \hat{U}(\mathbf{t}) \ket{\Phi_0}.
\end{equation}

It is of course possible to perform the optimization using gradient free optimizers (such as Nelder-Mead) or using finite difference, but it is usually preferable to evaluate the gradients $g_\mu$ of the energy as well.
Assuming a that $\hat{U}(\mathbf{t})$ generates only a real wave function, the gradients are given by

\begin{equation}
\begin{split}
g_\mu = \frac{\partial E_\mathrm{VQE}(\mathbf{t})}{\partial t_\mu} = &
2 \, \mathrm{Re} \bra{\Phi_0}  \hat{U}^\dagger(\mathbf{t}) \hat{H} \frac{\partial \hat{U}(\mathbf{t})}{\partial t_\mu}  \ket{\Phi_0}
\end{split}
\end{equation}

In QForte, running an algorithm is simple and can be done with just a few lines. First one instantiates an `algorithm` object by passing the molecule and reference lst. Then one calls `algorithm.run()` passing arguments that specify the run parameters.

> Run a dUCCSD-VQE calculation.

In [3]:
# Run the dUCCSD-VQE algorithm.
alg = UCCNVQE(LiHmol)
alg.run(opt_thresh=1.0e-2, pool_type='SD')
print(f'\n\n Efci:   {LiHmol.fci_energy:+12.10f}')

Warning, can't apply hamiltonain as tensor for fock computer, setting option to false

         Proceeding with totally symmetric.


-----------------------------------------------------
          Unitary Coupled Cluster VQE   
-----------------------------------------------------


               ==> UCCN-VQE options <==
---------------------------------------------------------
Trial reference state:                    |111100000000>
Number of Hamiltonian Pauli terms:        631
Trial state preparation method:           occupation_list
Trotter order (rho):                      1
Trotter number (m):                       1
Use fast version of algorithm:            True
Measurement variance thresh:              NA
Use qubit excitations:                    False
Use compact excitation circuits:          False
Optimization algorithm:                   BFGS
Optimization maxiter:                     200
Optimizer grad-norm threshold (theta):    1.00e-02
Use analytic gradient:               

### Example 2: SRQK

In a single reference quantum Krylov algorithm, a general state is written as a linear combination of the basis $\{ \psi_\alpha \}$ generated from the action of the (real) time evolution operator $e^{-i n \Delta t \hat{\mathcal{H}}}$ on a reference state $\ket{\Phi_0}$ as
\begin{equation}
\ket{\Psi} = \sum_n  c_n \ket{\psi_n} = \sum_{n=0}^{s} c_n e^{-i n \Delta t \hat{\mathcal{H}}} \ket{\Phi_0}. 
\end{equation}

Variational minimization of the energy of the state $\Psi$ leads to the following generalized eigenvalue problem
\begin{equation}
\label{eq:gep}
\mathbf{Hc} = \mathbf{Sc} E,
\end{equation}

where the elements of the overlap matrix ($\mathbf{S}$) and Hamiltonian ($\mathbf{H}$) are given by
\begin{align}
\label{eq:overlap}
S_{\alpha\beta} &= \langle \psi_\alpha | \psi_\beta \rangle
= \langle \Phi_{I} | \hat{U}^\dagger_m \hat{U}_n |\Phi_J \rangle,\\
\label{eq:hamiltonian}
H_{\alpha\beta} &= \langle \psi_\alpha | \hat{H} | \psi_\beta \rangle
= \langle \Phi_{I} | \hat{U}^\dagger_m \hat{H} \hat{U}_n |\Phi_{J} \rangle.
\end{align} 

This approach forms a basis which is very similar to a classical Krylov basis in the small time-step regime, and may also be thought of as a quantum version of classical filter diagonalization.

> Run a SRQK calculation.

In [8]:
# Run a single reference QK calculation.
# alg = SRQK(LiHmol)
# alg.run()

s = 5
dt = 0.1

r = 1
order = 1


alg = qf.SRQK(
    LiHmol,
    computer_type = 'fci',
    trotter_number=r,
    trotter_order=order,
    use_exact_evolution=False
    )

alg.run(
    s=s,
    dt=dt
    )

print(f'\n\n Efci:   {LiHmol.fci_energy:+12.10f}')

Warning, can't apply hamiltonain as tensor for fock computer, setting option to false

-----------------------------------------------------
           Single Reference Quantum Krylov   
-----------------------------------------------------


                     ==> QK options <==
-----------------------------------------------------------
Trial reference state:                    |111100000000>
Number of Hamiltonian Pauli terms:        631
Trial state preparation method:           occupation_list
Trotter order (rho):                      1
Trotter number (m):                       1
Use exact time evolution?:                False
Use fast version of algorithm:            True
Measurement varience thresh:              NA
Dimension of Krylov space (N):            6
Delta t (in a.u.):                        0.1
Target root:                              0



   k(S)            E(Npar)     N(params)           N(CNOT)          N(measure)
----------------------------------------------------